In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import dateutil.relativedelta
import math
import pygsheets
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from adaptation import ExcelReview

In [3]:
token_new_bi = 'HtdQ1pqETq'
token_cosm_bi = 'lpLLZsHACx'

current_date = datetime.today()
c_day = current_date.day
c_month = current_date.month
c_year = current_date.year

yesterday_date = current_date - dateutil.relativedelta.relativedelta(days=1)
y_day = yesterday_date.day
y_month = yesterday_date.month
y_year = yesterday_date.year


seven_month_date = current_date - dateutil.relativedelta.relativedelta(month=7)
s_day = seven_month_date.day
s_month = seven_month_date.month
s_year = seven_month_date.year

In [4]:
cat_str = "Баннер, АВИТО (Ручная подача), Авито (Покупка услуг), Авито (руч. подача), Авито - поднятие объявлений, Авито реклама, Баннер, Баннеры, Баннеры на колышках, Баннеры-г.Челябинск, Баннеры-ручное списание, Брендовая расклейка, Визитки, Визитки г. Пыть-Ях, Визитки-наклейки, Выгрузка за объявления на МЛСН, Выгрузка на сайты, Выгрузка объявлений на сайты (автовыгрузка), Дополнительный статус на Авито, За Баннеры, За баннеры, За визитки, За поднятие объявления на сайте Авито, За рекламу, За рекламу на АВИТО, За услуги расклейщика, Заказ баннера, Заказ баннеров, Заказ баннеров/фолий, Заказ визиток, Изготовление Баннеров-г.Когалым, Изготовление баннеров, Индивидуальная реклама, Использование баннера, Клей для расклейки, Листовки, Листовки , Маркетинг, Монтаж баннера, Наклейка на окно , Наклейка-визитка, Наклейки, Наклейки на авто, Наклейки на автомобиль, Наклейки на стекла, Объявление Авито, Объявление в газеты, Объявление на сайт N1, Объявление на сайт Авито , Объявление на сайты, Объявление на сайты , Объявление на сайты АвитоPro, Объявления в газеты, Объявления на АВИТО, Объявления на сайт 63.ru, Объявления на сайт АВИТО, Объявления на сайт Авито, Объявления на сайт Авито (ручная выгрузка), Объявления на сайт Из рук в руки, Объявления на сайт Сибдом, Объявления на сайт Циан (ручная выгрузка), Объявления на сайт ЯКТ, Объявления на сайте Avito, Объявления на сайты, Оплата размещения на авито, Оплата расклейщикам, Печать расклейки, Платная выгрузка, Применение платного статуса на Авито, Размещения объявлений на сайт авито, Расклейка, Расклейка объявлений , Расклейщик, Реклама, Реклама на АВИТО, Реклама на Авито, Рекламные листовки, Рекламные расходы, Служба сервиса: расклейка, Служба сервиса: фотографирование, Создание баннеров, Создание визиток, Создание наклеек на авто, Списание за баннер, Списание за визитки, Списание за объявления на сайт ЕМЛС, Списание за расклейку, Списание за рекламу на Авито, Списание на рекламу, Списание с бонусного счета (Авито), Строчная реклама, Услуги расклейщика, ЦИАН (ручная подача), Циан (ручная подача), баннеры, визитка в авто, визитки, доп.статусы на Авито, заказ баннера, заказ баннеров, заказ визиток, заказ наклеек, монтаж баннера, наклейка на автомобиль, наклейка на стекло, оплата баннеров, печать визиток, печать расклейки, платный статус, размещение объявлений в газету, расклейка"
cat_list = cat_str.split(', ')

req_zatrati_rekl = {
    "token": token_new_bi,
    "report_id": 393,
    "fields": ["city_name", "fio", "summ", "payment_date"],
    "filter_query":[
        {
            "field": "payment_date",
            "values": [f"{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}"],
            "type": "date"
        },
        {
            "field": "cat_name",
            "values": cat_list
        }
    ],
    "rows": 10000,
    "start": 0
}

def make_table_req_zatrati_rekl(table):
    table['payment_date'] = pd.to_datetime(table['payment_date'])
    table['date'] = table['payment_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

    table['summ'] = table['summ'].astype(float)
    end_table = table.pivot_table('summ', ['city_name', 'fio'], 'date', aggfunc="sum")
    end_table = end_table.fillna(0)
    #end_table['summing'] = end_table.sum(axis=1)
    end_table = end_table.reset_index()
    
    date_column = f"{y_year}-{y_month}-{y_day}"
    end_table[date_column] = end_table[date_column].astype(str)
    end_table[date_column] = end_table[date_column].apply(lambda x: x.replace('.', ','))

    return end_table

req_zatrati_rekl = ExcelReview(req_zatrati_rekl)
req_zatrati_rekl.df = make_table_req_zatrati_rekl(req_zatrati_rekl.get_table())

#req_zatrati_rekl.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'РекламаДень', 'a1')
req_zatrati_rekl.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'РекламаДень', 'fio', ['city_name'])

In [ ]:
del req_zatrati_rekl

In [14]:
zayv_type = 'Дистанционная покупка, Новостройки, Покупка вторичная, Покупка гаражи, Покупка загородной, Покупка загородной элитной недвижимости , Покупка коммерческой, Покупка элитная'
zayv_type = zayv_type.split(', ')

zayv_status = 'deposit, free, open'
zayv_status = zayv_status.split(', ')

zayv_reason = ', , Другое, Клиент отказался от фотографирования, На контроле у риэлтора, Нет ответа, Объект на задатке, Объект продан, Объект удален, Отмена, Продажа объекта приостановлена, Риэлтор отказался от фотографирования, Риэлтор самостоятельно сфотографировал объект, Семейные обстоятельства, выбран неверный город, консультация, купили/продали, не актуально (без причин), не доступен, не отвечает, нереальная потребность, нет подходящих объектов, объект находится далеко, отказались из-за комиссии, отложена, думают, передано другому риэлтору, передумали, по давности, сдали/сняли, сняли с другим АН'
zayv_reason = zayv_reason.split(', ')


zayv_na_pokup = {
    'token': token_new_bi,
    'report_id': 242,
    'fields': ['realtor', 'u_city', 'ticket_id'],
    'filter_query': [
        {
            'field': 'type_name',
            'values': zayv_type
        },
        {
            'field': 'status',
            'values': zayv_status
        }, 
        {
            'field': 'reason',
            'values': zayv_reason
        },
        {
            'field': 'realtor_start_date',
            "values": [f"{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}"],
            "type": "date"
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_table_zayv_na_pokup(table):
    table['ticket_id'] = table['ticket_id'].astype(str)
    
    table = table.pivot_table('ticket_id', ['realtor', 'u_city'], aggfunc='count')
    table = table.reset_index()
    date_column = f"{y_year}-{y_month}-{y_day}"
    table = table.rename(columns={'ticket_id':date_column})
    
    return table


zayv_na_pokup = ExcelReview(zayv_na_pokup)
zayv_na_pokup.df = make_table_zayv_na_pokup(zayv_na_pokup.get_table())
#zayv_na_pokup.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ЗаявкиНаПокупкуДень', 'a1')
zayv_na_pokup.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ЗаявкиНаПокупкуДень', 'realtor', ['u_city'])

In [20]:
temp_df['ticket_id'] = temp_df['ticket_id'].astype(str)
temp_df = temp_df.pivot_table('ticket_id', ['realtor', 'u_city'], aggfunc='count')
temp_df = temp_df.reset_index()
date_column = f"{y_year}-{y_month}-{y_day}"
temp_df = temp_df.rename(columns={'ticket_id':date_column})

In [21]:
temp_df

,realtor,u_city,2023-11-15
0,Aelita Muradian,Dubai,1
1,Akmal Shaimov,Dubai,1
2,Aleksandr Maslov (Александр Маслов),Dubai,1
3,Allahverdiyev Vidadi,Antalya,1
4,Anjelica Yakunina (Анжелика Якунина),Dubai,1
...,...,...,...
4765,тест Гореев,Калуга,1
4766,Қазтай Дархан Армантайұлы,Кокшетау,1
4767,Қалабеков Ерсұлтан Ерғалиұлы,Шымкент,1
4768,Құрбан Галия Ғалымжанқызы,Шымкент,1


In [ ]:
objects_in_base = {}

In [ ]:
podborki = {}

In [ ]:
pokazi_pokupatel = {}

In [ ]:
ipotech_consult = {}

In [ ]:
zayv_ipotek_bank = {}

In [ ]:
#load_by_day - обновляет таблицу на ежедневной основе
#load_full_table - загружает первый день в начале месяца
#для make_sort_table нужны поля из таблицы, которые влияют на merge

In [ ]:
#выкачать всю таблицу, сделать merge со всей таблицей
#т.к. в таблице сгрупированные данные выкачиваться и закачиваться будет не долго